In [6]:
import streamlit as st
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier

# from sklearn import model_selection
# from sklearn.linear_model import LogisticRegression
import pickle

In [3]:
st.write("""
# My first app
Hello *world!*
""")
 
# df = pd.read_csv("my_data.csv")
# st.line_chart(df)

In [5]:
# export processed data set to csv

# export features for tree-based models
X_train_comb = pd.read_csv('../data/petfinder-adoption-prediction/train/X_train_minmax_scaled_processed.csv')
X_test_comb =pd.read_csv('../data/petfinder-adoption-prediction/train/X_test_minmax_scaled_processed.csv')


# export target
y_train_comb = pd.read_csv('../data/petfinder-adoption-prediction/train/y_train.csv')
y_test_comb = pd.read_csv('../data/petfinder-adoption-prediction/train/y_test.csv')

gbc = GradientBoostingClassifier(n_estimators=200,subsample=1, max_leaf_nodes=31, max_features='log2', max_depth=5, loss = 'log_loss', learning_rate=0.025, random_state=42)
gbc.fit(X_train_comb,y_train_comb)

/Users/lanacasselmann/neuefische/ds_capstone_pet_adoptability/.venv/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(learning_rate=0.025, max_depth=5,
                           max_features='log2', max_leaf_nodes=31,
                           n_estimators=200, random_state=42, subsample=1)

In [7]:
# Save Model Using Pickle
# save the model to disk
filename = 'gbc.sav'
# methods for opening a file: "r" - Read - Default value. Opens a file for reading, error if the file does not exist
# "a" - Append - Opens a file for appending, creates the file if it does not exist
# "w" - Write - Opens a file for writing, creates the file if it does not exist
# "x" - Create - Creates the specified file, returns an error if the file exists
# In addition you can specify if the file should be handled as binary or text mode
# "t" - Text - Default value. Text mode
# "b" - Binary - Binary mode (e.g. images)
pickle.dump(gbc, open(filename, 'wb'))

In [10]:
X_train_comb.columns.to_list()

['type',
 'gender',
 'sterilized',
 'breed_type',
 'vaccinated_dewormed',
 'fee_bin',
 'maturitysize_0',
 'maturitysize_1',
 'maturitysize_2',
 'maturitysize_3',
 'furlength_0',
 'furlength_1',
 'furlength_2',
 'health_0',
 'health_1',
 'health_2',
 'color_pattern_0',
 'color_pattern_1',
 'color_pattern_2',
 'photoamt_11',
 'age_bin',
 'description_char']